<a href="https://colab.research.google.com/github/AryanSant27/Lora-Therapist/blob/main/Therapy_with_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers datasets peft accelerate bitsandbytes


In [ ]:
!pip install -U bitsandbytes

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "meta-llama/Llama-2-7b-hf"

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)



In [ ]:
from datasets import load_dataset

data = load_dataset("kunalchamoli/mental_health_v1")
train_data = data["train"]


In [ ]:
def format_example(example):
    instruction = example["instruction"].strip()
    output_text = example["output"].strip()

    # No need to add anything, instruction already has the required format
    prompt = instruction

    return {
        "text": prompt + "\n" + output_text  # for causal LM
        # "prompt": prompt, "label": output_text  # for seq2seq models
    }

# Apply to dataset
formatted_dataset = data.map(format_example)



In [ ]:
from transformers import AutoTokenizer

tokenizer.pad_token = tokenizer.eos_token
tokenized_data = formatted_dataset.map(lambda samples: tokenizer(samples["text"], truncation = True, padding = "max_length", max_length = 512), batched=True)


In [ ]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, TaskType

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()






In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./checkpoints",
    save_strategy="no",                     # <--- Don't save checkpoints at all during training
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    max_steps=65,
    learning_rate=2e-4,
    logging_steps=10,
    bf16=True,                               # or fp16=True if no bf16 support
    optim="paged_adamw_8bit",
    report_to="none",
    gradient_checkpointing=False,            # <--- No gradient checkpointing (disable memory tricks that can be unstable)
    save_total_limit=0,
    label_names=["labels"]# <--- No checkpoint limit needed (no saves)
)



In [ ]:
from transformers import Trainer, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    data_collator=data_collator,
)

In [ ]:
trainer.train()

In [ ]:
!pip install safetensors


In [ ]:
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")


In [ ]:
from google.colab import files
files.download("saved_model.zip")
